## Feature Selection
In this notebook, we will perform Data Cleansing, Feature Selection and Feature Engineering on the `Train_Data` information that we have stored in the IBM Object store as part of the previous step of Extract, Transform and Load (ETL).

In [1]:
import numpy as np # import numpy library
import pandas as pd

from pyspark.sql import SparkSession

pd.set_option("display.max_columns", None)

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191012143431-0000
KERNEL_ID = 300b673c-10a7-4c28-a206-1809c08dffea


load saved data from IBM Data Store

In [2]:
import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-1ffa7090-b46d-409d-a4c2-d72608483d6a',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token',
    'api_key': 'rJ9gWSE1VMXj0qVCzTfP36owXVp9NrU3vaRpVRtcgyz3'
}

configuration_name = 'os_e69b22554ae141c3a275fabb55b0f50e_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
train_data = spark.read.parquet(cos.url('Train_Data.parquet', 'advanceddatasciencewithibm-donotdelete-pr-z8s5dzzkvq4bck'))
train_data.createOrReplaceTempView("train_data")

# display first 5 rows, with scroll like in Pandas df
train_data.limit(5).toPandas().head()


,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,299215,8,145,9305,3,15,5,8,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
1,377492,26,619,11605,2,15,7,5,t,r,n,f,q,s,d,0,1,0,0,0,0,1,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0,1
2,294802,20,385,10589,2,15,11,6,o,w,q,f,q,s,d,0,1,1,0,0,0,1,1,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0,3
3,843787,21,873,11921,3,45,4,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,0,1,1,0,0,0,0,0,0,0,0,0,3
4,343815,17,421,9044,3,20,15,8,t,r,q,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3


In [3]:
# how many rows & columns? 
print((train_data.count(), len(train_data.columns)))

(260601, 40)


In [4]:
# show column types
train_data.dtypes

[('building_id', 'int'),
 ('geo_level_1_id', 'int'),
 ('geo_level_2_id', 'int'),
 ('geo_level_3_id', 'int'),
 ('count_floors_pre_eq', 'int'),
 ('age', 'int'),
 ('area_percentage', 'int'),
 ('height_percentage', 'int'),
 ('land_surface_condition', 'string'),
 ('foundation_type', 'string'),
 ('roof_type', 'string'),
 ('ground_floor_type', 'string'),
 ('other_floor_type', 'string'),
 ('position', 'string'),
 ('plan_configuration', 'string'),
 ('has_superstructure_adobe_mud', 'int'),
 ('has_superstructure_mud_mortar_stone', 'int'),
 ('has_superstructure_stone_flag', 'int'),
 ('has_superstructure_cement_mortar_stone', 'int'),
 ('has_superstructure_mud_mortar_brick', 'int'),
 ('has_superstructure_cement_mortar_brick', 'int'),
 ('has_superstructure_timber', 'int'),
 ('has_superstructure_bamboo', 'int'),
 ('has_superstructure_rc_non_engineered', 'int'),
 ('has_superstructure_rc_engineered', 'int'),
 ('has_superstructure_other', 'int'),
 ('legal_ownership_status', 'string'),
 ('count_families',

### Rename the damage_grade column to label

In [5]:
train_data = train_data.withColumnRenamed('damage_grade', 'label')
train_data.limit(5).toPandas().head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,label
0,299215,8,145,9305,3,15,5,8,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
1,377492,26,619,11605,2,15,7,5,t,r,n,f,q,s,d,0,1,0,0,0,0,1,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0,1
2,294802,20,385,10589,2,15,11,6,o,w,q,f,q,s,d,0,1,1,0,0,0,1,1,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0,3
3,843787,21,873,11921,3,45,4,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,0,1,1,0,0,0,0,0,0,0,0,0,3
4,343815,17,421,9044,3,20,15,8,t,r,q,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3


### Bin `Age` Variable
We have seen from a previous boxplot that `Age` has an extreme outlier (Value `995`) and values range from 1 to 995.  To overcome the outlier effect and varying range, suggest to create bins for the `Age` variable

In [6]:
from pyspark.ml.feature import Bucketizer

splits = [-1, 5 , 10, 25, 50, 100, float("inf")]

bucketizer = Bucketizer(splits=splits, inputCol="age", outputCol="age_group")

# Transform original data into its bucket index.
train_data = bucketizer.transform(train_data)
train_data.limit(5).toPandas().head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,label,age_group
0,299215,8,145,9305,3,15,5,8,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3,2.0
1,377492,26,619,11605,2,15,7,5,t,r,n,f,q,s,d,0,1,0,0,0,0,1,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0,1,2.0
2,294802,20,385,10589,2,15,11,6,o,w,q,f,q,s,d,0,1,1,0,0,0,1,1,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0,3,2.0
3,843787,21,873,11921,3,45,4,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,0,1,1,0,0,0,0,0,0,0,0,0,3,3.0
4,343815,17,421,9044,3,20,15,8,t,r,q,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3,2.0


In [7]:
# show counts by the age groups
train_data.groupBy('age_group').count().show()

+---------+------+
|age_group| count|
+---------+------+
|      0.0| 26041|
|      1.0| 33697|
|      4.0| 21913|
|      3.0| 68374|
|      2.0|107088|
|      5.0|  3488|
+---------+------+



We also noticed that the `has_secondary_use` variables are binary having either `0` or `1`.  We will create a new column `secondary_use` to collapse these secondary use variables.  First, we will initiailise column to `0` and next update the column to one of the following values:
- 10 when `has_secondary_use_agriculture` is `1`,
- 9  when `has_secondary_use_hotel` is `1`,
- 8  when `has_secondary_use_rental` is `1`,
- 7  when `has_secondary_use_institution` is `1`,
- 6  when `has_secondary_use_school` is `1`,
- 5  when `has_secondary_use_industry` is `1`,
- 4  when `has_secondary_use_health_post` is `1`,
- 3  when `has_secondary_use_gov_office` is `1`,
- 2  when `has_secondary_use_use_police` is `1`,
- 1  when `has_secondary_use_other` is `1`,


In [8]:
from pyspark.sql.functions import lit, when
from pyspark.sql import functions as F

sec_use_var = [i[0] for i in train_data.dtypes if (i[0].startswith('has_secondary_use_'))]
                                                                   
train_data = train_data.withColumn('secondary_use_code', lit(0))

i= 1
for col in reversed(sec_use_var):
    train_data = train_data.withColumn('secondary_use_code',F.when(train_data[col]==1,i).otherwise(train_data['secondary_use_code']))   
    i = i  + 1

In [9]:
# show counts by the age groups
train_data.groupBy('secondary_use_code').count().show()

+------------------+------+
|secondary_use_code| count|
+------------------+------+
|                 1|   777|
|                 6|    94|
|                 3|    38|
|                 5|   279|
|                 9|  8763|
|                 4|    49|
|                 8|  2111|
|                 7|   245|
|                10| 16777|
|                 2|    23|
|                 0|231445|
+------------------+------+



Remove the columns for which we have binned (`age`) or have created a representative variable (all `has_secondary_use` variables)

In [10]:
# create the list of columns to be dropped
drop_cols=sec_use_var[:]

drop_cols.extend(['has_secondary_use','age'])

#train_data.drop(*drop_cols).collect()
train_data = train_data.select([col for col in train_data.columns if col not in drop_cols]) 

In the dataset, there are also many categorical variables that has single character string values.  The distinct types for the variables  as seen from the boxplots in the data exploration step. These categorical variables are good candidates to one-hot encoding.

In [11]:
train_data.limit(5).toPandas().head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,label,age_group,secondary_use_code
0,299215,8,145,9305,3,5,8,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,3,2.0,0
1,377492,26,619,11605,2,7,5,t,r,n,f,q,s,d,0,1,0,0,0,0,1,0,0,0,0,v,1,1,2.0,10
2,294802,20,385,10589,2,11,6,o,w,q,f,q,s,d,0,1,1,0,0,0,1,1,0,0,0,v,1,3,2.0,10
3,843787,21,873,11921,3,4,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,0,3,3.0,10
4,343815,17,421,9044,3,15,8,t,r,q,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,3,2.0,0


In [12]:
train_data.dtypes

[('building_id', 'int'),
 ('geo_level_1_id', 'int'),
 ('geo_level_2_id', 'int'),
 ('geo_level_3_id', 'int'),
 ('count_floors_pre_eq', 'int'),
 ('area_percentage', 'int'),
 ('height_percentage', 'int'),
 ('land_surface_condition', 'string'),
 ('foundation_type', 'string'),
 ('roof_type', 'string'),
 ('ground_floor_type', 'string'),
 ('other_floor_type', 'string'),
 ('position', 'string'),
 ('plan_configuration', 'string'),
 ('has_superstructure_adobe_mud', 'int'),
 ('has_superstructure_mud_mortar_stone', 'int'),
 ('has_superstructure_stone_flag', 'int'),
 ('has_superstructure_cement_mortar_stone', 'int'),
 ('has_superstructure_mud_mortar_brick', 'int'),
 ('has_superstructure_cement_mortar_brick', 'int'),
 ('has_superstructure_timber', 'int'),
 ('has_superstructure_bamboo', 'int'),
 ('has_superstructure_rc_non_engineered', 'int'),
 ('has_superstructure_rc_engineered', 'int'),
 ('has_superstructure_other', 'int'),
 ('legal_ownership_status', 'string'),
 ('count_families', 'int'),
 ('label

Form a pipeline to index Categorical variables, One-Hot Encode them, then normalize all variables.
Also, fit a Random Forest algorithm to extract the top 20 features

In [13]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoderEstimator, VectorAssembler,  VectorSlicer
from pyspark.ml.classification import  RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

encoding_var = [i[0] for i in train_data.dtypes if (i[1]=='string')]
num_var = [i[0] for i in train_data.dtypes if ((i[1]=='int') | (i[1]=='double')) & (i[0]!='label')]

stages = []
for col in encoding_var:
    # Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol = col, outputCol = 'IDX_' + col)
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=["OHE_" + col],handleInvalid='keep')
    stages += [stringIndexer, encoder]

assemblerInputs = ["OHE_" + c for c in encoding_var] + num_var
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed = 8464,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)
stages += [rf]

pipeline = Pipeline(stages = stages)
model = pipeline.fit(train_data)
train_proc = model.transform(train_data)


In [14]:
# Compute accuracy score
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(train_proc)
print("Full Features Accuracy = %g" % accuracy)

Full Features Accuracy = 0.603328


Extract top features (out of a total of 59) of the classification algorithm.  The feature importance score returned comes in the form of a sparse vector which will be ampped to obtain the actual variable names.

In [15]:
model.stages[-1].featureImportances

SparseVector(59, {0: 0.0001, 1: 0.0003, 2: 0.0001, 3: 0.2491, 4: 0.0048, 5: 0.0029, 6: 0.0823, 7: 0.0009, 8: 0.0013, 9: 0.0046, 10: 0.0785, 11: 0.0195, 12: 0.0057, 13: 0.0508, 14: 0.0002, 16: 0.0249, 17: 0.0047, 18: 0.0094, 19: 0.0331, 20: 0.0066, 21: 0.0029, 22: 0.0003, 23: 0.0, 24: 0.0, 25: 0.0, 26: 0.0, 35: 0.0002, 38: 0.0001, 39: 0.1762, 40: 0.0061, 41: 0.0007, 42: 0.0529, 43: 0.0063, 44: 0.0086, 45: 0.0004, 46: 0.0602, 47: 0.0106, 48: 0.0015, 49: 0.0084, 50: 0.0296, 51: 0.0125, 52: 0.0081, 53: 0.0002, 54: 0.011, 55: 0.0007, 56: 0.0042, 57: 0.0174, 58: 0.0011})

In [16]:
def ExtractFeatureImp(featureImp, dataset, featuresCol):
    list_extract = []
    for i in dataset.schema[featuresCol].metadata["ml_attr"]["attrs"]:
        list_extract = list_extract + dataset.schema[featuresCol].metadata["ml_attr"]["attrs"][i]
    varlist = pd.DataFrame(list_extract)
    varlist['score'] = varlist['idx'].apply(lambda x: featureImp[x])
    return(varlist.sort_values('score', ascending = False))

ExtractFeatureImp(model.stages[-1].featureImportances, train_proc, "features").head(25)


,idx,name,ord,vals,score
24,3,OHE_foundation_type_r,NaN,NaN,0.249065
1,39,geo_level_1_id,NaN,NaN,0.176188
27,6,OHE_foundation_type_i,NaN,NaN,0.082313
31,10,OHE_roof_type_x,NaN,NaN,0.078470
8,46,has_superstructure_mud_mortar_stone,NaN,NaN,0.060194
4,42,count_floors_pre_eq,NaN,NaN,0.052867
34,13,OHE_ground_floor_type_v,NaN,NaN,0.050800
40,19,OHE_other_floor_type_s,NaN,NaN,0.033075
12,50,has_superstructure_cement_mortar_brick,NaN,NaN,0.029590
37,16,OHE_other_floor_type_q,NaN,NaN,0.024868


We will now construct a new `features_top` column to contain the top 25 features and create a new input vector column with only these variables.

In [17]:
varlist = ExtractFeatureImp(model.stages[-1].featureImportances, train_proc, "features")

varidx = [x for x in varlist['idx'][0:25]]

varidx

[3,
 39,
 6,
 10,
 46,
 42,
 13,
 19,
 50,
 16,
 11,
 57,
 51,
 54,
 47,
 18,
 44,
 49,
 52,
 20,
 43,
 40,
 12,
 4,
 17]

In [18]:
slicer = VectorSlicer(inputCol="features", outputCol="features_top", indices=varidx)
train_proc = slicer.transform(train_proc)

In [19]:
train_proc = train_proc.drop('rawPrediction', 'probability', 'prediction')

rf2 = RandomForestClassifier(labelCol="label", featuresCol="features_top", seed = 8464,
                            numTrees=10, cacheNodeIds = True, subsamplingRate = 0.7)
model2 = rf2.fit(train_proc)
train_proc = model2.transform(train_proc)

In [20]:
# Compute accuracy score
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(train_proc)
print("Top Features Accuracy = %g" % accuracy)

Top Features Accuracy = 0.57755


Prediction accuracy has dropped about 2.6% from reducing the full set of features to the top 25 features.  For the next stage of model building, we will keep all the features instead of just the top features since the accuracy has not increased when less features is used.

In [22]:
train_proc = train_proc.select(['building_id','features','label']) 


In [23]:
train_proc.dtypes

[('building_id', 'int'), ('features', 'vector'), ('label', 'int')]

## Store Feature Engineered Data in IBM Object Store
Let us go ahead and persist our feature engineered data into the IBM Object store for us to be able to use it in the next Model Building, Training and Deployment step.

In [24]:
train_proc = train_proc.repartition(1)
train_proc.write.mode('overwrite').parquet(cos.url('Train_Proc.parquet', 'advanceddatasciencewithibm-donotdelete-pr-z8s5dzzkvq4bck'))

Read back and check if the persisted data is good. 

In [25]:
train_proc = spark.read.parquet(cos.url('Train_Proc.parquet', 'advanceddatasciencewithibm-donotdelete-pr-z8s5dzzkvq4bck'))
print('Number of rows/columns read = ', (train_proc.count(), len(train_proc.columns)))
train_proc.limit(5).toPandas().head()

Number of rows/columns read =  (260601, 3)


,building_id,features,label
0,299215,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",3
1,377492,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",1
2,294802,"(0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",3
3,843787,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...",3
4,343815,"(1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3
